# AVISO : Génération du datacube

<div class="alert alert-danger">
Attention ce tutoriel repose sur l'ancienne version de Pangeo-Forge, une mise à jour majeur de la librairie rend le code obsolète à patir de la version 0.10
</div>

Utiliser l'environnement: *pangeo-forge-recipes-0.9-env*

In [1]:
import os
import intake
import fsspec
import logging
import xarray as xr
import pandas as pd
from glob import glob
from pathlib import Path

from pangeo_forge_recipes.patterns import FilePattern, ConcatDim
from pangeo_forge_recipes.recipes import HDFReferenceRecipe
from pangeo_forge_recipes.storage import (
    CacheFSSpecTarget,
    FSSpecTarget,
    MetadataTarget,
    StorageConfig,
)

In [2]:
# Chemin vers les données AVISO
fs = fsspec.filesystem('file')
data_root = Path("/home/ref-cmems-public/tac/sea-level/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D")

## Découverte des jeux de données

In [3]:
# Année Max et Min du jeu de donnée AVISO
aviso_folder = fs.ls(data_root)
years = sorted([int(element.split('/')[-1]) for element in aviso_folder if fs.isdir(element)])
min_year, max_year = min(years), max(years)
display(min_year, max_year)

1993

2023

In [4]:
# Jour min de l'année min
min_year_folder = fs.ls(data_root / str(min_year))
days = sorted([int(element.split('/')[-1]) for element in min_year_folder if fs.isdir(element)])
min_day = days[0]
display(min_year, min_day)

1993

1

In [5]:
# Jour max de l'année max
max_year_folder = fs.ls(data_root / str(max_year))
days = sorted([int(element.split('/')[-1]) for element in max_year_folder if fs.isdir(element)])
max_day = days[-1]
display(max_year, max_day)

2023

158

les données vont du 01/01/1993 (1993,1) au 07/06/2023 (2023, 158)

In [6]:
files = fs.glob(f"{data_root}/1993/00[1-9]/*.nc")
[os.path.basename(file) for file in files][:10]

['dt_global_allsat_phy_l4_19930101_20210726.nc',
 'dt_global_allsat_phy_l4_19930102_20210726.nc',
 'dt_global_allsat_phy_l4_19930103_20210726.nc',
 'dt_global_allsat_phy_l4_19930104_20210726.nc',
 'dt_global_allsat_phy_l4_19930105_20210726.nc',
 'dt_global_allsat_phy_l4_19930106_20210726.nc',
 'dt_global_allsat_phy_l4_19930107_20210726.nc',
 'dt_global_allsat_phy_l4_19930108_20210726.nc',
 'dt_global_allsat_phy_l4_19930109_20210726.nc']

On peux voir que le chemin consiste:
<ul>
    <li style='color:blue;'>une racine constante</li>
    <li style='color:red;'>un dossier pour l'année (4-digit)</li>
    <li style='color:orange;'>un dossier pour le jour de l'année (3-digit)</li>
    <li style='color:green;'>un nom de fichier finissant par .nc</li>
</ul>

par exemple : <p><span style="color: blue;">*/home/ref-cmems-public/tac/sea-level/SEALEVEL_GLO_PHY_L4_MY_008_047/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D/</span><span style="color: red;">1993/</span><span style="color: orange;">001/</span><span style="color: green;">dt_global_allsat_phy_l4_19930101_20210726.nc*</span></p>

In [7]:
# Etude de la taille des chunks natifs des fichiers NetCDF
import netCDF4
nc = netCDF4.Dataset(files[0], 'r')
for var_name, var in nc.variables.items():
    print(f"Variable: {var_name}, Taille des chunks: {var.chunking()}")

Variable: crs, Taille des chunks: contiguous
Variable: time, Taille des chunks: [1]
Variable: latitude, Taille des chunks: [50]
Variable: lat_bnds, Taille des chunks: [50, 2]
Variable: longitude, Taille des chunks: [50]
Variable: lon_bnds, Taille des chunks: [50, 2]
Variable: nv, Taille des chunks: [2]
Variable: sla, Taille des chunks: [1, 50, 50]
Variable: err_sla, Taille des chunks: [1, 50, 50]
Variable: ugosa, Taille des chunks: [1, 50, 50]
Variable: err_ugosa, Taille des chunks: [1, 50, 50]
Variable: vgosa, Taille des chunks: [1, 50, 50]
Variable: err_vgosa, Taille des chunks: [1, 50, 50]
Variable: adt, Taille des chunks: [1, 50, 50]
Variable: ugos, Taille des chunks: [1, 50, 50]
Variable: vgos, Taille des chunks: [1, 50, 50]
Variable: flag_ice, Taille des chunks: [1, 50, 50]
Variable: tpa_correction, Taille des chunks: [1]


## Définition des fonctions

In [8]:
def create_recipe(name, dates, path_format, output):
    
    def make_path(time):
        return glob(path_format.format(time=time))[0]

    time_concat_dim = ConcatDim("time", dates)
    pattern = FilePattern(make_path, time_concat_dim)
    recipe = HDFReferenceRecipe(pattern)
    
    # Création des dossiers
    target_path = output / 'references' / name 
    metadata_path = output / 'metadata' / name
    cache_path = output / 'cache' / name
    
    fs, _, _ = fsspec.get_fs_token_paths(target_path)
    fs.mkdirs(target_path, exist_ok=True)
    target = FSSpecTarget(fs=fs, root_path=target_path)


    fs, _, _ = fsspec.get_fs_token_paths(metadata_path)
    if fs.exists(metadata_path):
        fs.rm(metadata_path, recursive=True)
    fs.mkdirs(metadata_path, exist_ok=True)
    metadata = MetadataTarget(fs=fs, root_path=metadata_path)


    fs, _, _ = fsspec.get_fs_token_paths(cache_path)
    if fs.exists(cache_path):
        fs.rm(cache_path, recursive=True)
    fs.mkdirs(cache_path, exist_ok=True)
    cache = CacheFSSpecTarget(fs=fs, root_path=cache_path)

    recipe.storage_config = StorageConfig(target=target, cache=cache, metadata=metadata)
    
    return recipe

In [9]:
def execute_recipe(recipe):
    task = recipe.to_dask()
    task.compute()

## Création de la recipe

In [62]:
name = 'aviso'
path_format = (
    "/home/ref-cmems-public/tac/sea-level/SEALEVEL_GLO_PHY_L4_MY_008_047/"
    "cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D/"
    "{time.year:04d}/{time.dayofyear:03d}/dt_global_allsat_phy_l4_{time:%Y%m%d}_*.nc"
)

wrk = Path('/home1/scratch/gcaer/data') # Dossier de travail où enregistrer les données du Datacube. 
version = 'datacube-year'
output = wrk / name / version

In [63]:
start, end = "2015-01-01", "2017-12-31"
#start, end = "1993-01-01", "2023-06-07"
dates = pd.date_range(start, end, freq="d")
groups = dates.groupby(dates.year)

In [52]:
recipes = dict()
for year, v in groups.items():
    output_year = output / str(year)
    recipes[year] = create_recipe(name, v, path_format, output_year)

## Exécuter la recipe

In [53]:
from distributed import Client
client = Client()

/home1/datawork/gcaer/conda-env/pangeo-forge-recipes-0.9-env/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60741 instead
  warnings.warn(


In [54]:
%%time
for year in recipes.keys():
    print(year)
    execute_recipe(recipes[year])

2015
CPU times: user 2.65 s, sys: 712 ms, total: 3.36 s
Wall time: 18.4 s


## Création du catalogue

In [55]:
def set_catalog(groups, output, name):
    import yaml
    
    # Création du catalogue intake
    sources = dict()
    for year in groups.keys():
        sources[str(year)] = {
            'args': {
                'chunks': {},
                'consolidated': False,
                'storage_options': {
                    'fo': f'{output.as_posix()}/{year}/references/{name}/reference.json',
                    'remote_options': {},
                    'remote_protocol': 'file',
                    'skip_instance_cache': True,
                    'target_options': {},
                    'target_protocol': 'file'
                },
                'urlpath': 'reference://'
            },
            'description': '',
            'driver': 'intake_xarray.xzarr.ZarrSource'
        }

    config = {
        'sources':sources
    }
    # Convertir les données en format YAML
    yaml_data = yaml.dump(config)

    # Écrire les données YAML dans un fichier
    with open(output / 'reference.yaml', 'w') as fichier_yaml:
        fichier_yaml.write(yaml_data)

In [56]:
set_catalog(groups, output, name)

## Ouvrir le datacube

In [57]:
catalog = output / "reference.yaml"
cat = intake.open_catalog(catalog)

In [58]:
%%time
_drop = ["crs","lat_bnds","lon_bnds","ugosa","err_ugosa","vgosa","err_vgosa","ugos","vgos","flag_ice","tpa_correction","nv",]
datacubes = []
for year in range(pd.to_datetime(start).year, pd.to_datetime(end).year + 1):
    datacube = cat[f'{year}'](chunks={"time": 1, "latitude": -1, "longitude": -1}).to_dask().drop_vars(_drop)
    datacubes.append(datacube)

CPU times: user 1.45 s, sys: 204 ms, total: 1.66 s
Wall time: 1.62 s


/home1/datawork/gcaer/conda-env/pangeo-forge-recipes-0.9-env/lib/python3.12/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


In [59]:
%%time
datacube = xr.concat(datacubes, dim="time")

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.23 ms


In [60]:
datacube.isel(time=slice(0, 20))

<xarray.Dataset>
Dimensions:    (time: 20, latitude: 720, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-01-20
Data variables:
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    err_sla    (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               2015-01-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             2014-12-31T12:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...